In [1]:
import os
import cv2
import tkinter as tk
import pandas as pd
from PIL import Image, ImageTk

VIDEO_PATH = "D:/Szakdolgozat/githubclone/kozlekedesi_tabla_felismero/Videos/0002.mp4"
CSV_PATH = "D:/Szakdolgozat/githubclone/kozlekedesi_tabla_felismero/Detections/0002.csv"
IMG_SIZE = (960, 540)  # Megjelenített méret

df = pd.read_csv(CSV_PATH)
columns = df.columns.tolist()
timestamp_col = columns[0]
label_cols = columns[1:]

cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)

# User Interface
class PredictionViewer:
    def __init__(self, root):
        self.root = root
        self.root.title("Modell predikció ellenőrző")

        self.canvas = tk.Canvas(root, width=IMG_SIZE[0], height=IMG_SIZE[1])
        self.canvas.grid(row=0, column=0, columnspan=3)

        self.info_label = tk.Label(root, text="", font=("Arial", 12), justify="left")
        self.info_label.grid(row=1, column=0, columnspan=3, pady=10)

        self.back_btn = tk.Button(root, text="<< Vissza", command=self.prev_frame)
        self.back_btn.grid(row=2, column=0)

        self.forward_btn = tk.Button(root, text="Előre >>", command=self.next_frame)
        self.forward_btn.grid(row=2, column=2)

        self.frame_index = 0
        self.update_frame()

    def update_frame(self):
        row = df.iloc[self.frame_index]
        timestamp = row[timestamp_col]
        cap.set(cv2.CAP_PROP_POS_MSEC, timestamp * 1000)
        ret, frame = cap.read()
        if not ret:
            return
        frame = cv2.resize(frame, IMG_SIZE)
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        imgtk = ImageTk.PhotoImage(image=img)
        self.canvas.imgtk = imgtk
        self.canvas.create_image(0, 0, anchor='nw', image=imgtk)

        detected = [name for name in label_cols if row[name] == 1]
        text = f"🕒 Idő: {timestamp:.1f} s\n📋 Felismert táblák: {', '.join(detected) if detected else 'nincs'}"
        self.info_label.config(text=text)

    def next_frame(self):
        if self.frame_index + 1 < len(df):
            self.frame_index += 1
            self.update_frame()

    def prev_frame(self):
        if self.frame_index > 0:
            self.frame_index -= 1
            self.update_frame()

if __name__ == "__main__":
    root = tk.Tk()
    app = PredictionViewer(root)
    root.mainloop()
